# m14 Cassandra  

### Import packages

In [2]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [15]:
clstr = Cluster()
session = clstr.connect()

NoHostAvailable: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})

### Use session to 'talk' to cassandra

In [4]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [5]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [6]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [7]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a warehouse table for testing

In [8]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    sku TEXT PRIMARY KEY, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
);
""")

### Load data

In [9]:
df = pd.read_csv('m14.csv')
df

,sku,name,description,warehouse_num
0,SKU101,Smartphone Model X,Latest smartphone with AI capabilities,1
1,SKU102,Laptop Model Y,High-performance laptop for professionals,2
2,SKU103,Headphones Model Z,Wireless noise-canceling headphones,1
3,SKU104,Camera Model Alpha,Digital camera with 4K video recording,3
4,SKU105,Tablet Model Beta,10-inch tablet for entertainment and productivity,2


In [10]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = SKU101, name =0, description = Latest smartphone with AI capabilities, warehouse_num = 1
sku = SKU102, name =1, description = High-performance laptop for professionals, warehouse_num = 2
sku = SKU103, name =2, description = Wireless noise-canceling headphones, warehouse_num = 1
sku = SKU104, name =3, description = Digital camera with 4K video recording, warehouse_num = 3
sku = SKU105, name =4, description = 10-inch tablet for entertainment and productivity, warehouse_num = 2


In [11]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('{row[0]}', '{row[1]}', '{row[2]}', {row[3]});
        """
       )


        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES (SKU101, Smartphone Model X, Latest smartphone with AI capabilities, 1);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES (SKU102, Laptop Model Y, High-performance laptop for professionals, 2);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES (SKU103, Headphones Model Z, Wireless noise-canceling headphones, 1);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES (SKU104, Camera Model Alpha, Digital camera with 4K video recording, 3);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES (SKU105, Tablet Model Beta, 10-inch tablet for entertainment and productivity, 2);
        


In [12]:
rows = session.execute("select (sku, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=SKU103, name=Headphones Model Z, description=Wireless noise-canceling headphones, warehouse_num=1
sku=SKU105, name=Tablet Model Beta, description=10-inch tablet for entertainment and productivity, warehouse_num=2
sku=SKU104, name=Camera Model Alpha, description=Digital camera with 4K video recording, warehouse_num=3
sku=SKU101, name=Smartphone Model X, description=Latest smartphone with AI capabilities, warehouse_num=1
sku=SKU102, name=Laptop Model Y, description=High-performance laptop for professionals, warehouse_num=2


In [13]:
session.execute("""CREATE INDEX ON m14.inventory(warehouse_num);""")

In [14]:
rows = session.execute("select (sku, name, description) from m14.inventory where warehouse_num = 1 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

SKU103, Headphones Model Z, Wireless noise-canceling headphones
SKU101, Smartphone Model X, Latest smartphone with AI capabilities
